<a href="https://colab.research.google.com/github/Menon-Vineet/Python-Codes/blob/main/ims_bearing_anomaly_detection_refactored.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unsupervised Bearing Anomaly Detection (IMS / NASA PCoE)

**Author:** Vineet Menon (el jefe)  
**Date:** 2026-01-19  
**Goal:** I compare classical unsupervised anomaly detectors (via **PyCaret**) against a lightweight **BiLSTM Autoencoder** on the IMS run-to-failure bearing vibration dataset.

---

## What you get in this notebook

- A **Colab-ready** workflow: install → download → featurize → train → score → visualize
- Self-contained utilities (no missing local scripts)
- Reproducible runs (seeded) and clean, readable code with docstrings + type hints

> Dataset download source: NASA Prognostics Center of Excellence (PCoE) “Bearings” ZIP on PHM S3.


## 0. Environment setup (Colab)

If you run this in Google Colab, execute the next cell once to install dependencies.


In [ ]:
# I keep installs in one place so the notebook is easy to share and re-run on Colab.
# Note: PyCaret pulls a sizable dependency tree; first install may take a bit.

%%capture
!pip -q install -U "pycaret[full]" pandas numpy matplotlib scikit-learn tqdm joblib
!pip -q install -U torch --index-url https://download.pytorch.org/whl/cpu


## 1. Imports and global configuration

In [ ]:
from __future__ import annotations

import os
import zipfile
import shutil
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import List, Optional, Dict

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

# PyCaret (anomaly)
from pycaret.anomaly import setup as anomaly_setup
from pycaret.anomaly import create_model, predict_model, save_model

# Torch (BiLSTM autoencoder)
import torch
from torch import nn

# I set seeds so results are repeatable across runs (as much as possible for CPU training).
SEED: int = 1234
np.random.seed(SEED)
torch.manual_seed(SEED)

# I store all artifacts under one project folder so Colab + GitHub runs remain tidy.
PROJECT_DIR = Path.cwd() / "ims_anomaly_project"
DATA_DIR = PROJECT_DIR / "data"
ARTIFACTS_DIR = PROJECT_DIR / "artifacts"
PLOTS_DIR = PROJECT_DIR / "plots"

for d in (DATA_DIR, ARTIFACTS_DIR, PLOTS_DIR):
    d.mkdir(parents=True, exist_ok=True)

print("Project dir:", PROJECT_DIR)


## 2. Utility functions

I keep the notebook self-contained by implementing the dataset download, parsing, feature extraction, and plotting helpers directly here.


In [ ]:
from typing import Dict, List

NASA_BEARINGS_ZIP_URL: str = "https://phm-datasets.s3.amazonaws.com/NASA/4.+Bearings.zip"

def download_file(url: str, dst_path: Path, overwrite: bool = False) -> Path:
    """Download a file to a local path.

    Args:
        url: Public URL to download.
        dst_path: Where I want to store the file.
        overwrite: If True, I re-download even if the file exists.

    Returns:
        The path to the downloaded file.
    """
    dst_path.parent.mkdir(parents=True, exist_ok=True)

    if dst_path.exists() and not overwrite:
        print(f"I am reusing existing file: {dst_path}")
        return dst_path

    # I use urllib from the standard library to keep this notebook dependency-light.
    import urllib.request

    print(f"I am downloading dataset to: {dst_path}")
    urllib.request.urlretrieve(url, dst_path)
    return dst_path


def unzip(zip_path: Path, extract_to: Path, overwrite: bool = False) -> Path:
    """Extract a ZIP archive.

    Args:
        zip_path: Path to a .zip file.
        extract_to: Directory to extract into.
        overwrite: If True, I delete the existing folder first.

    Returns:
        The extraction directory.
    """
    if extract_to.exists() and overwrite:
        shutil.rmtree(extract_to)

    extract_to.mkdir(parents=True, exist_ok=True)

    # I explicitly open and extract to avoid shelling out and to keep it cross-platform.
    with zipfile.ZipFile(zip_path, "r") as zf:
        zf.extractall(extract_to)

    return extract_to


def parse_ims_filename_to_datetime(filename: str) -> datetime:
    """Parse IMS file timestamps like '2003.10.22.12.06.24' into a datetime."""
    stem = Path(filename).stem
    return datetime.strptime(stem, "%Y.%m.%d.%H.%M.%S")


def find_test_folder(root: Path, test_name: str) -> Path:
    """Locate a test folder (e.g., '1st_test', '2nd_test', '3rd_test') inside the extracted dataset."""
    candidates = list(root.rglob(test_name))
    if not candidates:
        raise FileNotFoundError(f"I could not find '{test_name}' under {root}")
    return candidates[0]


def load_ims_timeseries(
    test_folder: Path,
    bearing_col: int = 0,
    max_files: Optional[int] = None,
) -> pd.DataFrame:
    """Load raw IMS vibration files into a time-indexed dataframe.

    Important:
        Each file is a 1-second vibration snapshot sampled at high frequency.
        Loading *every sample* can be huge, so I typically convert each file into one compact feature row.

    Args:
        test_folder: Folder containing timestamp-named vibration files.
        bearing_col: Which column/channel I load (0-based).
        max_files: If set, I only load the first N files (useful for quick demos).

    Returns:
        DataFrame with a single column 'signal' (numpy arrays) indexed by timestamp.
    """
    files = sorted([p for p in test_folder.iterdir() if p.is_file()])
    if max_files is not None:
        files = files[:max_files]

    rows: List[pd.Series] = []
    idx: List[pd.Timestamp] = []

    for fp in tqdm(files, desc=f"Loading raw signal col={bearing_col}"):
        ts = parse_ims_filename_to_datetime(fp.name)
        try:
            arr = pd.read_csv(fp, header=None, delim_whitespace=True).iloc[:, bearing_col].to_numpy(dtype=float)
        except Exception as e:
            raise RuntimeError(f"I failed reading file {fp}: {e}") from e

        # I store the entire 1-second snapshot as a numpy array object; downstream I will featurize it.
        rows.append(pd.Series({"signal": arr}))
        idx.append(pd.Timestamp(ts))

    df = pd.DataFrame(rows, index=idx).sort_index()
    df.index.name = "timestamp"
    return df


def rms(x: np.ndarray) -> float:
    """Root-mean-square, a standard vibration energy feature."""
    return float(np.sqrt(np.mean(np.square(x), dtype=float)))


def peak_to_peak(x: np.ndarray) -> float:
    """Peak-to-peak amplitude."""
    return float(np.max(x) - np.min(x))


def kurtosis_excess(x: np.ndarray) -> float:
    """Excess kurtosis (kurtosis - 3) without requiring scipy."""
    x = x.astype(float)
    mu = x.mean()
    s2 = np.mean((x - mu) ** 2)
    if s2 == 0:
        return 0.0
    m4 = np.mean((x - mu) ** 4)
    return float(m4 / (s2 ** 2) - 3.0)


def featurize_snapshot(signal: np.ndarray) -> Dict[str, float]:
    """Convert one raw vibration snapshot into a compact set of statistical features."""
    # I keep features simple, fast, and interpretable.
    return {
        "rms": rms(signal),
        "mean": float(np.mean(signal)),
        "std": float(np.std(signal)),
        "p2p": peak_to_peak(signal),
        "kurtosis_excess": kurtosis_excess(signal),
    }


def featurize_timeseries(raw_df: pd.DataFrame) -> pd.DataFrame:
    """Featurize a dataframe produced by `load_ims_timeseries` into numeric columns."""
    feats = [featurize_snapshot(sig) for sig in tqdm(raw_df["signal"].to_list(), desc="Featurizing snapshots")]
    return pd.DataFrame(feats, index=raw_df.index)


def plot_series(df: pd.DataFrame, columns: List[str], title: str, fname: str) -> None:
    """Plot time series columns and save under the project plots folder."""
    plt.figure(figsize=(12, 4))
    for col in columns:
        plt.plot(df.index, df[col], label=col)

    plt.title(title)
    plt.xlabel("timestamp")
    plt.legend()
    plt.tight_layout()

    out = PLOTS_DIR / f"{fname}.png"
    plt.savefig(out, dpi=150)
    plt.show()
    print("Saved:", out)


## 3. Download + extract the dataset

The IMS bearing dataset ships as a ZIP containing folders like `1st_test`, `2nd_test`, and `3rd_test`.


In [ ]:
zip_path = DATA_DIR / "NASA_4_Bearings.zip"
extract_root = DATA_DIR / "NASA_4_Bearings"

# I download once and reuse locally (handy for iterative work).
download_file(NASA_BEARINGS_ZIP_URL, zip_path, overwrite=False)
unzip(zip_path, extract_root, overwrite=False)

print("Extracted to:", extract_root)
print("Top-level contents:", [p.name for p in extract_root.iterdir()][:10])


## 4. Load one test run and build features

To keep this notebook runnable on modest machines, I show a fast path:
- Load **one channel** from a test folder
- Convert each 1-second snapshot into a **feature row** (RMS, p2p, etc.)

You can scale to more channels/features once the pipeline is working.


In [ ]:
TEST_NAME = "2nd_test"        # try: "1st_test", "2nd_test", "3rd_test"
BEARING_COL = 0                 # vibration channel column index
MAX_FILES = 800                 # I cap files for a quick demo; set None for full run (slower)

test_folder = find_test_folder(extract_root, TEST_NAME)
print("Using test folder:", test_folder)

raw_df = load_ims_timeseries(test_folder, bearing_col=BEARING_COL, max_files=MAX_FILES)
feat_df = featurize_timeseries(raw_df)

print("Feature dataframe shape:", feat_df.shape)
feat_df.head()


## 5. Quick visualization

RMS is often a good first “health indicator” proxy for bearings (it tends to rise as damage accumulates).


In [ ]:
plot_series(
    feat_df,
    ["rms"],
    title=f"{TEST_NAME} - RMS over time (channel {BEARING_COL})",
    fname="rms_over_time",
)


## 6. Train/test split (time-aware)

Because this is *run-to-failure*, I treat the early segment as “mostly normal” and score later segments as potential anomalies.


In [ ]:
TRAIN_FRACTION = 0.6

n = len(feat_df)
cut = int(n * TRAIN_FRACTION)

train_df = feat_df.iloc[:cut].copy()
test_df = feat_df.iloc[cut:].copy()

print("Train rows:", len(train_df), "| Test rows:", len(test_df))


## 7. PyCaret anomaly models

I use PyCaret to quickly compare multiple unsupervised detectors.

Notes:
- PyCaret expects purely numeric columns (our feature set is numeric by design)
- I use the **train segment** to fit the model, then score the tail segment


In [ ]:
_ = anomaly_setup(
    data=train_df,
    session_id=SEED,
    normalize=True,
    silent=True,
    html=False,
)

ANOMALY_MODELS: List[str] = ["iforest", "knn", "lof", "svm", "mcd"]
pycaret_results: Dict[str, pd.DataFrame] = {}

for model_name in ANOMALY_MODELS:
    print("\n" + "=" * 80)
    print(f"I am training PyCaret anomaly model: {model_name}")

    model = create_model(model_name)
    scored = predict_model(model, data=test_df)

    # I persist the pipeline so the notebook is shareable and reusable.
    save_model(model, str(ARTIFACTS_DIR / f"{model_name}_pipeline"))

    pycaret_results[model_name] = scored

print("\nSaved model pipelines under:", ARTIFACTS_DIR)


### 7.1 Visualize anomaly scores (example)

PyCaret returns:
- `Anomaly` (0/1 flag)
- `Anomaly_Score` (higher usually means “more anomalous”, model-dependent)


In [ ]:
MODEL_TO_PLOT = "iforest"

scored = pycaret_results[MODEL_TO_PLOT].copy()
print("Columns:", list(scored.columns))

plot_df = scored.join(test_df[["rms"]], how="left")

plot_series(
    plot_df,
    ["rms", "Anomaly_Score"],
    title=f"{MODEL_TO_PLOT} - RMS vs Anomaly_Score",
    fname=f"{MODEL_TO_PLOT}_score_vs_rms",
)

anom_rate = float(plot_df["Anomaly"].mean() * 100)
print(f"Anomaly rate (flagged=1) for {MODEL_TO_PLOT}: {anom_rate:.2f}%")

plot_df.head()


## 8. BiLSTM Autoencoder (PyTorch)

Why this model:
- Sequence models can learn temporal structure across time
- Autoencoders give a natural **reconstruction error** as an anomaly score

Here, I use a compact BiLSTM autoencoder on **scaled feature vectors**.


In [ ]:
@dataclass(frozen=True)
class SequenceConfig:
    """Configuration for creating sliding windows."""

    window: int = 32
    stride: int = 1


def make_windows(x: np.ndarray, cfg: SequenceConfig) -> np.ndarray:
    """Create overlapping windows from a 2D array (time, features).

    Args:
        x: Array of shape (T, F).
        cfg: Window/stride configuration.

    Returns:
        Windows of shape (N, window, F).
    """
    if x.ndim != 2:
        raise ValueError("I expected x to be 2D: (time, features).")

    T, _ = x.shape
    if T < cfg.window:
        raise ValueError(f"I need at least {cfg.window} rows, but got {T}.")

    windows = []
    for start in range(0, T - cfg.window + 1, cfg.stride):
        windows.append(x[start : start + cfg.window])

    return np.stack(windows, axis=0)


class BiLSTMAutoencoder(nn.Module):
    """A small BiLSTM autoencoder for sequence reconstruction."""

    def __init__(self, n_features: int, hidden: int = 32) -> None:
        super().__init__()
        self.encoder = nn.LSTM(
            input_size=n_features,
            hidden_size=hidden,
            num_layers=1,
            batch_first=True,
            bidirectional=True,
        )
        self.decoder = nn.LSTM(
            input_size=2 * hidden,
            hidden_size=hidden,
            num_layers=1,
            batch_first=True,
            bidirectional=True,
        )
        self.proj = nn.Linear(2 * hidden, n_features)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # I encode the input sequence, then decode it back to the original feature dimension.
        z, _ = self.encoder(x)
        y, _ = self.decoder(z)
        return self.proj(y)


def train_autoencoder(
    model: nn.Module,
    x_train: np.ndarray,
    epochs: int = 8,
    batch_size: int = 64,
    lr: float = 1e-3,
) -> List[float]:
    """Train the autoencoder and return epoch losses."""
    device = torch.device("cpu")
    model.to(device)
    model.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.SmoothL1Loss()  # Huber-like loss; robust to spikes.

    dataset = torch.utils.data.TensorDataset(torch.tensor(x_train, dtype=torch.float32))
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    epoch_losses: List[float] = []
    for ep in range(1, epochs + 1):
        losses = []
        for (xb,) in loader:
            xb = xb.to(device)

            optimizer.zero_grad()
            recon = model(xb)
            loss = loss_fn(recon, xb)
            loss.backward()
            optimizer.step()

            losses.append(float(loss.detach().cpu().item()))

        mean_loss = float(np.mean(losses))
        epoch_losses.append(mean_loss)
        print(f"Epoch {ep:02d}/{epochs} | loss={mean_loss:.6f}")

    return epoch_losses


def reconstruction_scores(model: nn.Module, x: np.ndarray, batch_size: int = 128) -> np.ndarray:
    """Compute per-window reconstruction error scores."""
    device = torch.device("cpu")
    model.eval()
    model.to(device)

    dataset = torch.utils.data.TensorDataset(torch.tensor(x, dtype=torch.float32))
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)

    scores: List[float] = []
    loss_fn = nn.SmoothL1Loss(reduction="none")

    with torch.no_grad():
        for (xb,) in loader:
            xb = xb.to(device)
            recon = model(xb)

            per_elem = loss_fn(recon, xb)
            per_window = per_elem.mean(dim=(1, 2)).detach().cpu().numpy()
            scores.extend(per_window.tolist())

    return np.array(scores, dtype=float)


### 8.1 Train + score with BiLSTM AE

I fit on the “mostly normal” early segment, then score the later segment.


In [ ]:
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_df)
test_scaled = scaler.transform(test_df)

cfg = SequenceConfig(window=32, stride=1)

train_w = make_windows(train_scaled, cfg)
test_w = make_windows(test_scaled, cfg)

print("Train windows:", train_w.shape, "| Test windows:", test_w.shape)

model = BiLSTMAutoencoder(n_features=train_w.shape[-1], hidden=32)
losses = train_autoencoder(model, train_w, epochs=8, batch_size=64, lr=1e-3)

plt.figure(figsize=(8, 3))
plt.plot(losses)
plt.title("BiLSTM AE training loss")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.tight_layout()
plt.show()

train_scores = reconstruction_scores(model, train_w)
test_scores = reconstruction_scores(model, test_w)

THRESH_PCTL = 90
threshold = float(np.percentile(train_scores, THRESH_PCTL))
print(f"Threshold (p{THRESH_PCTL}) = {threshold:.6f}")

test_flags = (test_scores >= threshold).astype(int)
print("Flagged anomalies in test windows:", int(test_flags.sum()), "out of", len(test_flags))


### 8.2 Align window scores back to timestamps

Window scores start after `window-1` steps, so I align them to the end of each window.


In [ ]:
test_index = test_df.index[cfg.window - 1 :]
bilstm_df = pd.DataFrame(
    {"recon_score": test_scores, "anomaly": test_flags},
    index=test_index,
)

plot_series(
    bilstm_df.join(test_df[["rms"]].iloc[cfg.window - 1 :]),
    ["rms", "recon_score"],
    title="BiLSTM AE - RMS vs reconstruction score",
    fname="bilstm_score_vs_rms",
)

print("Sample flagged timestamps:")
bilstm_df[bilstm_df["anomaly"] == 1].head(10)


## 9. Next steps

- Add more channels and richer features (spectral bands, envelope, crest factor)
- Use a time-aware evaluation protocol (lead time, early-warning horizon)
- Persist the scaler + AE weights under `artifacts/` for deployment

For GitHub, I recommend adding a `requirements.txt` and a short `README.md` that cites the dataset source.
